In [ ]:
#company_list = ['TTEF.PA','SHEL.L','REP.MC','EQNR.OL','ENI.MI','BP.L','2222.SE','NESTE.HE','GALP.LS','CVX','XOM.N','VLO.N','GAZP.MM','PSX','MPC.N']
#company_list = ["TSLA.O","F","GM","STLAM.MI","MBGn.DE","002594.SZ","RACE.MI","BMWG.DE","VOWG.DE","7267.T","005380.KS","MRTI.NS","P911_p.DE","TAMO.NS"]
#company_list = ["JPM","BAC","601398.SS","WFC","601288.SS","601988.SS","601939.SS","HSBA.L","RY.TO","HDBK.NS","CBA.AX","8306.T","600036.SS","TD.TO","ICBK.NS","1120.SE","SBI.NS","BNPP.PA","8316.T","SAN.MC","BBCA.JK","SBER.MM","BMO.TO","NAB.AX","BBVA.MC"]
#company_list = ["BHP.AX","RIO.L","SCCO.K","FCX","GLEN.L","601899.SS","CRH","VALE3.SA","HOLN.S","FMG.AX","1211.SE","NUE","GMEXICOB.MX","NEM","AMMN.JK","MLM","VMC","ULTC.NS","AAL.L","AEM.TO","ABX.TO","JSTL.NS","ANTO.L","005490.KS"]
#company_list=["TMUS.O","600941.SS","VZ","CMCSA.O","T","DTEGn.DE","9432.T","BRTI.NS","9984.T","601728.SS","9433.T","9434.T","AMXB.MX","7010.SE","CHTR.O","EAND.AD","ORAN.PA","BCE.TO","SCMN.S","2412.TW","STEL.SI","TLS.AX","TEF.MC","VOD.L","T.TO"]

In [9]:
%matplotlib notebook
%matplotlib inline
%reload_ext autoreload
%autoreload 2

#import eikon as ek
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup
from textblob import TextBlob
import datetime
from datetime import time
import warnings
warnings.filterwarnings("ignore")
#ek.set_app_key('bd7b3cc97d0f46a6b55932ab99a4feab4875dda9')

In [11]:
import master_thesis_tool_kit as mtk

ModuleNotFoundError: No module named 'eikon'

In [3]:
sector_code = 501020
SCREENER_REQUEST = 'SCREEN(U(IN(Equity(active,public,primary))), TOP(TR.CompanyMarketCap , 9999, nnumber),\
IN(TR.TRBCIndustryGroupCode,'+str(sector_code)+'), TR.SocialPillarScore(Period=FY0)>=0, CURN=USD)'
fields = ["TR.CommonName","TR.CompanyMarketCap","TR.TRBCIndustryGroup","TR.SocialPillarScore(Period=FY0)"]
df1,err= ek.get_data(SCREENER_REQUEST, fields)

In [4]:
if df1.shape[0]>100:
    df1_sorted = df1.sort_values('Company Market Cap', ascending=False).head(100)
else:
    df1_sorted = df1.sort_values('Company Market Cap', ascending=False)
    
company_list = df1_sorted["Instrument"].tolist()
sector = ''.join(df1_sorted["TRBC Industry Group Name"].unique())

In [5]:
res = pd.DataFrame()
csv_file = os.path.join(f'Weights_{sector}_companies.csv')
if not os.path.exists(sector):
    os.makedirs(sector)

In [6]:
for company_ticker in company_list:
    df = mtk.company_headlines(company_ticker)
    if not df.empty:
        df_analyzed = mtk.comapny_sentiment_score(df)
        filepath = os.path.join(sector, f'df_analyzed_{company_ticker}.csv')
        df_analyzed.to_csv(filepath, header=True, index=False, sep=';')
        res = mtk.polarity_to_weight(df_analyzed, company_ticker, res)

2024-04-12 14:58:02,284 P[14616] [MainThread 14112] Backend error. 503 Service Unavailable


EikonError: Error code 503 | Backend error. 503 Service Unavailable

In [ ]:
df1 = mtk.company_social_score(company_list)
join_df = res.set_index("Company").join(df1.set_index("Instrument"), how='left')
join_df.to_csv(csv_file, sep=";", index=True, header=True)

In [ ]:
output_file = 'aggregate_data.csv'
df_test = mtk.generate_aggregate_csv(output_file)